In [1]:
import pandas as pd
import numpy as np
import os
import glob
import json
from joblib import dump, load
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

## Configuration

In [2]:
# Number of force cells in the robotic leg
N_CELLS = 8

# Path where the results are stored
RESULTS_PATH = '../../../../results'
# ID of the training and validation data resulting from this notebook, stored in RESULTS_PATH
DATA_ID = '0006_30062021'
# Hyperparameters search date
HS_DATE = '01072021'
# Number of folds in cross-validation
CV = 6

print('Model trained with data: ' + DATA_ID)

pd.set_option('display.max_columns', None)

Model trained with data: 0006_30062021


## Hyperparameters seach analysis

In [25]:
results_files_ls = glob.glob(os.path.join(RESULTS_PATH, DATA_ID, 'RF_{}'.format(HS_DATE), 'RF_{}_*.json'.format(HS_DATE)))

print('Number of results files: {}'.format(len(results_files_ls)))

Number of results files: 788


In [26]:
# Load all the results and generates a pandas dataframe
results_ls = []
for results_file in results_files_ls:
    with open(results_file) as json_file:
        results_dict = json.load(json_file)
        
    dict_aux = {}
    dict_aux['params_ID'] = results_dict['id']
    for key, value in results_dict['parameters'].items():
        dict_aux['param_' + key] = value
    for key, value in results_dict['cv_results'].items():
        dict_aux['__'.join([key, 'mean'])] = np.mean(value)
        dict_aux['__'.join([key, 'std'])] = np.std(value)

    results_ls.append(dict_aux)
        
results_df = pd.DataFrame(results_ls)
results_df

,params_ID,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,fit_time__mean,fit_time__std,Train_Fx_MAE_mean__mean,Train_Fx_MAE_mean__std,Train_Fx_MAE_std__mean,Train_Fx_MAE_std__std,Train_Fx_MSE_mean__mean,Train_Fx_MSE_mean__std,Train_Fx_MSE_std__mean,Train_Fx_MSE_std__std,Train_Fx_R2_mean__mean,Train_Fx_R2_mean__std,Train_Fx_R2_std__mean,Train_Fx_R2_std__std,Train_Fy_MAE_mean__mean,Train_Fy_MAE_mean__std,Train_Fy_MAE_std__mean,Train_Fy_MAE_std__std,Train_Fy_MSE_mean__mean,Train_Fy_MSE_mean__std,Train_Fy_MSE_std__mean,Train_Fy_MSE_std__std,Train_Fy_R2_mean__mean,Train_Fy_R2_mean__std,Train_Fy_R2_std__mean,Train_Fy_R2_std__std,Train_Fz_MAE_mean__mean,Train_Fz_MAE_mean__std,Train_Fz_MAE_std__mean,Train_Fz_MAE_std__std,Train_Fz_MSE_mean__mean,Train_Fz_MSE_mean__std,Train_Fz_MSE_std__mean,Train_Fz_MSE_std__std,Train_Fz_R2_mean__mean,Train_Fz_R2_mean__std,Train_Fz_R2_std__mean,Train_Fz_R2_std__std,Valid_Fx_MAE_mean__mean,Valid_Fx_MAE_mean__std,Valid_Fx_MAE_std__mean,Valid_Fx_MAE_std__std,Valid_Fx_MSE_mean__mean,Valid_Fx_MSE_mean__std,Valid_Fx_MSE_std__mean,Valid_Fx_MSE_std__std,Valid_Fx_R2_mean__mean,Valid_Fx_R2_mean__std,Valid_Fx_R2_std__mean,Valid_Fx_R2_std__std,Valid_Fy_MAE_mean__mean,Valid_Fy_MAE_mean__std,Valid_Fy_MAE_std__mean,Valid_Fy_MAE_std__std,Valid_Fy_MSE_mean__mean,Valid_Fy_MSE_mean__std,Valid_Fy_MSE_std__mean,Valid_Fy_MSE_std__std,Valid_Fy_R2_mean__mean,Valid_Fy_R2_mean__std,Valid_Fy_R2_std__mean,Valid_Fy_R2_std__std,Valid_Fz_MAE_mean__mean,Valid_Fz_MAE_mean__std,Valid_Fz_MAE_std__mean,Valid_Fz_MAE_std__std,Valid_Fz_MSE_mean__mean,Valid_Fz_MSE_mean__std,Valid_Fz_MSE_std__mean,Valid_Fz_MSE_std__std,Valid_Fz_R2_mean__mean,Valid_Fz_R2_mean__std,Valid_Fz_R2_std__mean,Valid_Fz_R2_std__std
0,J2B8WWA60A,13,0.01,0.010,0.0001,500,1.628091,0.015680,9.633375,0.116903,3.379386,0.109681,183.452179,5.226086,133.790528,7.307454,0.436990,0.009628,0.116463,0.003827,7.693463,0.059237,4.444596,0.078595,167.484605,4.281618,178.595138,7.717352,0.346292,0.004293,0.061686,0.002588,15.916403,0.207966,6.151072,0.143095,502.570743,9.729472,437.966848,12.598493,0.336618,0.007068,0.157754,0.005198,10.238879,0.515371,3.780939,0.662211,208.003110,30.967329,162.135557,52.373896,0.350398,0.035872,0.146517,0.023933,8.103008,0.243362,4.774938,0.294386,187.424325,22.163635,220.114660,40.277787,0.263947,0.021040,0.093315,0.018450,16.918744,0.957662,6.756130,0.687986,570.952406,65.823592,512.361381,101.162489,0.216110,0.042635,0.216298,0.049153
1,0151K82W8D,7,0.05,0.100,0.0001,100,0.193335,0.005371,11.063047,0.145711,3.609706,0.155726,248.088821,6.810880,166.146076,10.309018,0.233048,0.004383,0.084722,0.001813,8.854367,0.074427,5.195281,0.072967,228.735606,4.553998,248.452396,6.194865,0.129941,0.004047,0.053113,0.002708,18.056416,0.233472,7.279606,0.135973,669.688494,14.868216,660.558996,24.718440,0.160206,0.003596,0.114593,0.003298,11.248976,0.656502,3.846822,0.715427,256.300132,33.671181,181.748385,57.846630,0.191288,0.020930,0.102938,0.020506,8.958685,0.345336,5.338321,0.227219,233.289750,22.876753,273.852186,31.276344,0.093555,0.023122,0.065363,0.015023,18.332648,1.108402,7.538675,0.697567,687.178268,80.526323,679.561239,144.473501,0.102899,0.038464,0.158383,0.045133
2,03TKM9QHB2,13,0.05,0.010,0.0001,1000,3.179395,0.016767,9.631186,0.116787,3.380743,0.110199,183.330471,5.271895,133.882957,7.332289,0.437482,0.009630,0.116896,0.003874,7.691312,0.058739,4.445027,0.079654,167.244094,4.407054,178.358857,7.912230,0.347411,0.004598,0.061832,0.002450,15.914497,0.209780,6.150102,0.143707,502.414202,9.778289,437.733301,12.349140,0.336793,0.006972,0.157994,0.005161,10.235621,0.516107,3.782166,0.660774,207.822916,30.966901,162.153486,52.420460,0.351019,0.036524,0.146973,0.023832,8.100066,0.240972,4.773637,0.295441,187.272752,22.105987,219.980871,40.124878,0.264784,0.021377,0.093705,0.018867,16.913470,0.962320,6.749786,0.689486,570.837150,66.064478,512.209594,100.804959,0.216149,0.043457,0.216808,0.049031
3,04859

In [27]:
# Sum up the scores by force axis in only one sortable score
for subset in ['Train', 'Valid']:
    for loss in ['MAE', 'MSE', 'R2']:
        results_df[subset + '_' + loss] = results_df[[subset + '_' + force + '_' + loss + '_mean__mean' for force in ['Fx', 'Fy', 'Fz']]].mean(axis=1)

In [28]:
# Sort the dataframe by the most relevant score
results_df = results_df.sort_values(['Valid_R2'], ascending=False)
results_df

,params_ID,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,fit_time__mean,fit_time__std,Train_Fx_MAE_mean__mean,Train_Fx_MAE_mean__std,Train_Fx_MAE_std__mean,Train_Fx_MAE_std__std,Train_Fx_MSE_mean__mean,Train_Fx_MSE_mean__std,Train_Fx_MSE_std__mean,Train_Fx_MSE_std__std,Train_Fx_R2_mean__mean,Train_Fx_R2_mean__std,Train_Fx_R2_std__mean,Train_Fx_R2_std__std,Train_Fy_MAE_mean__mean,Train_Fy_MAE_mean__std,Train_Fy_MAE_std__mean,Train_Fy_MAE_std__std,Train_Fy_MSE_mean__mean,Train_Fy_MSE_mean__std,Train_Fy_MSE_std__mean,Train_Fy_MSE_std__std,Train_Fy_R2_mean__mean,Train_Fy_R2_mean__std,Train_Fy_R2_std__mean,Train_Fy_R2_std__std,Train_Fz_MAE_mean__mean,Train_Fz_MAE_mean__std,Train_Fz_MAE_std__mean,Train_Fz_MAE_std__std,Train_Fz_MSE_mean__mean,Train_Fz_MSE_mean__std,Train_Fz_MSE_std__mean,Train_Fz_MSE_std__std,Train_Fz_R2_mean__mean,Train_Fz_R2_mean__std,Train_Fz_R2_std__mean,Train_Fz_R2_std__std,Valid_Fx_MAE_mean__mean,Valid_Fx_MAE_mean__std,Valid_Fx_MAE_std__mean,Valid_Fx_MAE_std__std,Valid_Fx_MSE_mean__mean,Valid_Fx_MSE_mean__std,Valid_Fx_MSE_std__mean,Valid_Fx_MSE_std__std,Valid_Fx_R2_mean__mean,Valid_Fx_R2_mean__std,Valid_Fx_R2_std__mean,Valid_Fx_R2_std__std,Valid_Fy_MAE_mean__mean,Valid_Fy_MAE_mean__std,Valid_Fy_MAE_std__mean,Valid_Fy_MAE_std__std,Valid_Fy_MSE_mean__mean,Valid_Fy_MSE_mean__std,Valid_Fy_MSE_std__mean,Valid_Fy_MSE_std__std,Valid_Fy_R2_mean__mean,Valid_Fy_R2_mean__std,Valid_Fy_R2_std__mean,Valid_Fy_R2_std__std,Valid_Fz_MAE_mean__mean,Valid_Fz_MAE_mean__std,Valid_Fz_MAE_std__mean,Valid_Fz_MAE_std__std,Valid_Fz_MSE_mean__mean,Valid_Fz_MSE_mean__std,Valid_Fz_MSE_std__mean,Valid_Fz_MSE_std__std,Valid_Fz_R2_mean__mean,Valid_Fz_R2_mean__std,Valid_Fz_R2_std__mean,Valid_Fz_R2_std__std,Train_MAE,Train_MSE,Train_R2,Valid_MAE,Valid_MSE,Valid_R2
111,NZO5TE3B4L,13,0.05,0.001,0.0001,10000,43.780256,0.383255,7.677225,0.105979,2.700615,0.088058,114.687254,3.408697,85.776329,4.228359,0.645787,0.009466,0.096529,0.002411,6.079501,0.062695,3.343031,0.090165,95.155414,3.075837,95.230762,5.946895,0.585452,0.005064,0.071672,0.002682,12.452204,0.200666,4.547755,0.132292,291.771144,6.514146,201.701731,5.292893,0.581715,0.008151,0.140559,0.006634,9.795947,0.385915,3.741132,0.593070,190.400700,25.896013,156.721497,43.107825,0.404354,0.058445,0.171829,0.029027,7.593492,0.276075,4.407584,0.397541,162.533029,26.810420,190.174401,56.513626,0.344356,0.021445,0.116253,0.027182,16.005167,0.663544,6.181631,0.487822,495.155921,38.586024,388.197268,56.114377,0.271685,0.054570,0.262390,0.067865,8.736310,167.204604,0.604318,11.131535,282.696550,0.340132
669,IJL38Z9NAQ,13,0.01,0.001,0.0010,10000,43.949214,0.291127,7.677225,0.105979,2.700615,0.088058,114.687254,3.408697,85.776329,4.228359,0.645787,0.009466,0.096529,0.002411,6.079501,0.062695,3.343031,0.090165,95.155414,3.075837,95.230762,5.946895,0.585452,0.005064,0.071672,0.002682,12.452204,0.200666,4.547755,0.132292,291.771144,6.514146,201.701731,5.292893,0.581715,0.008151,0.140559,0.006634,9.795947,0.385915,3.741132,0.593070,190.400700,25.896013,156.721497,43.107825,0.404354,0.058445,0.171829,0.029027,7.593492,0.276075,4.407584,0.397541,162.533029,26.810420,190.174401,56.513626,0.344356,0.021445,0.116253,0.027182,16.005167,0.663544,6.181631,0.487822,495.155921,38.586024,388.197268,56.114377,0.271685,0.054570,0.262390,0.067865,8.736310,167.204604,0.604318,11.131535,282.696550,0.340132
228,L6BHB49WUS,13,0.05,0.001,0.0010,10000,44.035617,0.363289,7.677225,0.105979,2.700615,0.088058,114.687254,3.408697,85.776329,4.228359,0.645787,0.009466,0.096529,0.002411,6.079501,0.062695,3.343031,0.090165,95.155414,3.075837,95.230762,5.946895,0.585452,0.005064,0.071672,0.002682,12.452204,0.200666,4.547755,0.132292,291.771144,6.514146,201.701731,5.292893,0.581715,0.008151,0.140559,0.006634,9.795947,0.385915,3.741132,0.593070,190.400700,25.896013,156.721497,43.107825,0.404354,0.058445,0.171829,0.029027,7.593492,0.276075,4.407584,0.397541,162.533029,26.810420,190.17440

In [29]:
best_params = dict(results_df.iloc[0][[col for col in results_df.columns if 'param_' in col]])
best_params = {key.replace('param_', ''): value for key, value in best_params.items()}
print('Best parameters: {}'.format(best_params))

Best parameters: {'max_depth': 13, 'max_features': 0.05, 'min_samples_leaf': 0.001, 'min_samples_split': 0.0001, 'n_estimators': 10000}


## Best model

In [8]:
# Load data
X_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_train_{}.npy'.format(DATA_ID)))
X_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_test_{}.npy'.format(DATA_ID)))
Y_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_train_{}.npy'.format(DATA_ID)))
Y_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_test_{}.npy'.format(DATA_ID)))

In [10]:
# Setup the model with the best parameters
model = RandomForestRegressor(**best_params, random_state=0, n_jobs=-1, verbose=1)

model.fit(X_train, Y_train)

# Save the model
dump(model, os.path.join(RESULTS_PATH, DATA_ID, 'RF_{}'.format(HS_DATE), 'RF_best_model_{}_{}.joblib'.format(HS_DATE, DATA_ID))) 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    5.0s finished


['../../../results/0003_11042021/RF_12042021/RF_best_model_12042021_0003_11042021.joblib']

In [11]:
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

results = {
    'Train': {
        'MAE': mean_absolute_error(Y_train, train_preds, multioutput='raw_values'),
        'MSE': mean_squared_error(Y_train, train_preds, multioutput='raw_values'),
        'R2': r2_score(Y_train, train_preds, multioutput='raw_values')
    },
    'Test': {
        'MAE': mean_absolute_error(Y_test, test_preds, multioutput='raw_values'),
        'MSE': mean_squared_error(Y_test, test_preds, multioutput='raw_values'),
        'R2': r2_score(Y_test, test_preds, multioutput='raw_values')
    }       
    
}

# # Display the score for each axis of each force cell
# for subset in ['Train', 'Test']:
#     for f, force in enumerate(['Fx', 'Fy', 'Fz']):
#         for c in range(N_CELLS):
#             for loss in ['MAE', 'MSE', 'R2']:
#                 scores = [results[subset][loss][i + f] for i in range(0, N_CELLS * 3, 3)]
#                 print('{} {}{}{} {}: {:.4f}'.format(subset, force[0], c + 1, force[-1], loss, scores[c]))
# print('\n')

# Display the score mean and standard deviation of each axis
for subset in ['Train', 'Test']:
    for f, force in enumerate(['Fx', 'Fy', 'Fz']):
        for loss in ['MAE', 'MSE', 'R2']:
            scores = [results[subset][loss][i + f] for i in range(0, N_CELLS * 3, 3)]
            print(' '.join([subset, force, loss]) + ': {:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished


Train Fx MAE: 8.6461 ± 2.4048
Train Fx MSE: 151.6214 ± 77.8937
Train Fx R2: 0.5655 ± 0.0612
Train Fy MAE: 8.7525 ± 6.0748
Train Fy MSE: 238.7282 ± 265.7849
Train Fy R2: 0.4389 ± 0.1117
Train Fz MAE: 11.2891 ± 3.9985
Train Fz MSE: 300.2048 ± 246.7115
Train Fz R2: 0.5458 ± 0.0617
Test Fx MAE: 13.0032 ± 5.0952
Test Fx MSE: 311.5999 ± 249.7830
Test Fx R2: 0.3473 ± 0.2863
Test Fy MAE: 10.7406 ± 7.7562
Test Fy MSE: 378.7206 ± 481.6318
Test Fy R2: 0.3404 ± 0.1027
Test Fz MAE: 19.0024 ± 6.5255
Test Fz MSE: 687.2684 ± 593.5296
Test Fz R2: 0.3611 ± 0.2479


In [ ]:
# plt.figure(figsize=(20,15))
# plt.scatter(Y_train[:, 3], Y_train[:, 4], label='true', alpha=0.3)
# plt.scatter(train_preds[:, 3], train_preds[:, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()

# plt.figure(figsize=(20,15))
# plt.scatter(Y_train[:100, 3], Y_train[:100, 4], label='true', alpha=0.3)
# plt.scatter(train_preds[:100, 3], train_preds[:100, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()

# plt.figure(figsize=(20,15))
# plt.scatter(Y_test[:, 3], Y_test[:, 4], label='true', alpha=0.3)
# plt.scatter(test_preds[:, 3], test_preds[:, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()